# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [39]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import plotly.express as px 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date, timedelta
from dotenv import load_dotenv
import os
import json

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [40]:
CITIES = ["Paris", "Berlin", "Madrid", "Amsterdam", "Vienna"]

In [41]:
### 💡 Première requête guidée

load_dotenv()
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5"

def get_weather_data(city):
    
    # URL : current weather data
    url = f"{OPENWEATHER_BASE_URL}/weather"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric',  # Celsius
        'lang': 'fr'
    }
    
    # On définit la requête
    r = requests.get(url, params)

    # Try de la requête
    try:
        requests.get(url, params)
    except:
        print(r.status_code)
    
    # Si la requête a aboutie, on print le coeur de la réponse
    print(r.json())
    

In [42]:
for i in CITIES: 
    get_weather_data(i)

{'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'ciel dégagé', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 20.59, 'feels_like': 20.48, 'temp_min': 19.88, 'temp_max': 22.27, 'pressure': 1020, 'humidity': 68, 'sea_level': 1020, 'grnd_level': 1011}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 270}, 'clouds': {'all': 0}, 'dt': 1753779027, 'sys': {'type': 1, 'id': 6550, 'country': 'FR', 'sunrise': 1753762797, 'sunset': 1753817639}, 'timezone': 7200, 'id': 2988507, 'name': 'Paris', 'cod': 200}
{'coord': {'lon': 13.4105, 'lat': 52.5244}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'partiellement nuageux', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 20.11, 'feels_like': 20.03, 'temp_min': 18.81, 'temp_max': 21.85, 'pressure': 1013, 'humidity': 71, 'sea_level': 1013, 'grnd_level': 1008}, 'visibility': 10000, 'wind': {'speed': 5.14, 'deg': 270}, 'clouds': {'all': 40}, 'dt': 1753779227, 'sys': {'type': 2, 

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

- Si l'API retourne une erreur 401, cela correspond à une erreur de type "unauthenticated". Cela veut dire qu'on n'a pas renseigné notre clé API dans l'appel, ou que notre clé est invalide. Il faut donc renseigner notre clé ou créer une nouvelle clé valide.
- Une ville introuvable devrait nous renvoyer un status code 404 "not found".

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [43]:
# Configuration Visual Crossing
VCH_API_KEY = os.getenv("VCH_API_KEY")
VCH_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
start_date = "2025-07-22"
end_date = "2025-07-28"

def get_historical_weather(city, start, end):

    url = f"{VCH_BASE_URL}/{city}/{start}/{end}"

    params = {
        'key': VCH_API_KEY,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure',
        'unitGroup': 'metric'
    }
   
    # On définit la requête
    r = requests.get(url, params)

    # Try de la requête
    try:
        requests.get(url, params)
    except:
        print(r.status_code)

    # Si la requête a aboutie, on récupère les données qui nous intéresse
    data_raw = r.json()
    data = data_raw.get("days", "clé inexistante")

    # Création DataFrame
    df = pd.DataFrame(data)
    df.insert(0,'city', city)
    return df

    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

In [44]:
for i in CITIES: 
    get_historical_weather(i, start_date, end_date)

### 🎯 Mission pratique
Récupérez les données des 30 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [45]:
list_df = [get_historical_weather(i, start_date, end_date) for i in CITIES]
df_cities = [pd.concat(list_df, copy=False)]
df_cities = df_cities[0]
print(df_cities)

        city    datetime  temp  humidity  precip  windspeed  pressure
0      Paris  2025-07-22  19.2      71.0   0.052       16.8    1013.4
1      Paris  2025-07-23  19.2      74.3   2.018       13.5    1013.8
2      Paris  2025-07-24  18.4      83.8  12.440       15.4    1014.9
3      Paris  2025-07-25  20.7      72.8   0.010       12.2    1017.7
4      Paris  2025-07-26  21.4      67.6   0.000       16.0    1018.2
5      Paris  2025-07-27  18.8      78.8   5.600       23.2    1016.4
6      Paris  2025-07-28  18.9      65.9   0.000       13.8    1019.9
0     Berlin  2025-07-22  17.5      86.8   6.500       23.2    1006.1
1     Berlin  2025-07-23  18.4      83.8  27.700       25.0    1008.0
2     Berlin  2025-07-24  19.5      80.5   0.600       16.6    1011.3
3     Berlin  2025-07-25  20.0      84.7   4.200       19.4    1013.5
4     Berlin  2025-07-26  19.4      78.4   2.500       18.4    1014.8
5     Berlin  2025-07-27  20.4      69.7   0.000       18.4    1010.7
6     Berlin  2025-0

In [46]:
### 🗺️ Obtenir les coordonnées
GEOCODING_API = "http://api.openweathermap.org/geo/1.0/direct"

def get_city_coordinates(city):
    
    # URL : current weather data
    url = f"{GEOCODING_API}"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'lang': "fr"
    }
    # On définit la requête
    r = requests.get(url, params)

    # Try de la requête
    try:
        requests.get(url, params)
    except:
        print(r.status_code)
    
    # Si la requête a aboutie, on récupère les coordonnées
    data_raw = r.json()
    data = [data_raw[0].get("lat"),data_raw[0].get("lon")]
    return data
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """

In [47]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5"
start_date_time = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
end_date_time = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())

def get_air_quality(city, start, end):

    # URL : pollution datas
    url = f"{OPENWEATHER_BASE_URL}/air_pollution/history"

    # Paramètres à compléter
    params = {
        'lat': get_city_coordinates(city)[0],
        'lon': get_city_coordinates(city)[1],
        'start': start,
        'end': end,
        'appid': OPENWEATHER_API_KEY,
        'lang': 'fr'
    }
    
    # On définit la requête
    r = requests.get(url, params)

    # Try de la requête
    try:
        requests.get(url, params)
    except:
        print(r.status_code)

    # Si la requête a aboutie, on stock la réponse dans une variable
    data_raw = r.json()
    data = data_raw.get("list")

    # On stock la longueur de la liste de la requête dans une variable list_data
    list_data = [d for d in range(0, len(data))]
    
    # On récupère le dictionnaire des components pour chaque jour de la requête
    air_data = [data[d].get("components") for d in list_data]
    
    # On récupère l'indice aqi pour chaque jour de la requête
    indice_aqi = [data[d].get("main").get("aqi") for d in list_data]
    
    # On récupère le timestamp pour chaque jour de la requête et on reconvertit en datetime
    time_data = [datetime.fromtimestamp(data[d].get("dt")).strftime("%Y-%m-%d") for d in list_data]
    
    
    # On intègre l'indice aqi et le time au dictionnaire de components de chaque
    for d,i in zip(air_data, list_data):
        d["indice aqi"] = indice_aqi[i]
        d["datetime"] = time_data[i]
    
    # Création du DataFrame
    air_df = pd.DataFrame(air_data)
    air_df.insert(0,'city', city)
    air_df = air_df.drop_duplicates(subset="datetime")
    air_df = air_df.reset_index(drop=True)
    return air_df

    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """

    

In [48]:
get_air_quality(CITIES[0], start_date_time, end_date_time)

,city,co,no,no2,o3,so2,pm2_5,pm10,nh3,indice aqi,datetime
0,Paris,105.69,0.0,1.03,67.04,0.23,1.85,2.66,1.62,2,2025-07-22
1,Paris,97.12,0.0,2.61,43.04,0.14,1.57,2.54,3.43,1,2025-07-23
2,Paris,109.45,0.0,2.17,45.27,0.09,1.07,1.20,0.94,1,2025-07-24
3,Paris,111.92,0.0,2.57,54.77,0.18,2.26,2.50,1.81,1,2025-07-25
4,Paris,112.99,0.0,4.14,50.84,0.19,4.37,5.17,3.33,1,2025-07-26
5,Paris,108.78,0.0,2.03,44.46,0.09,1.47,2.13,2.46,1,2025-07-27
6,Paris,121.22,0.0,1.54,71.94,0.55,2.74,3.57,2.68,2,2025-07-28


In [49]:
list_poll = [get_air_quality(i, start_date_time, end_date_time) for i in CITIES]
df_poll = [pd.concat(list_poll, copy=False)]
df_poll = df_poll[0]

In [87]:
df_total = pd.concat([df_cities, df_poll], axis=1)
df_total = df_total.iloc[:,:-1]
df_total = df_total.reset_index(drop=True)
df_total = df_total.loc[:, ~df_total.columns.duplicated()]
df_total

,city,datetime,temp,humidity,precip,windspeed,pressure,co,no,no2,o3,so2,pm2_5,pm10,nh3,indice aqi
0,Paris,2025-07-22,19.2,71.0,0.052,16.8,1013.4,105.69,0.00,1.03,67.04,0.23,1.85,2.66,1.62,2
1,Paris,2025-07-23,19.2,74.3,2.018,13.5,1013.8,97.12,0.00,2.61,43.04,0.14,1.57,2.54,3.43,1
2,Paris,2025-07-24,18.4,83.8,12.440,15.4,1014.9,109.45,0.00,2.17,45.27,0.09,1.07,1.20,0.94,1
3,Paris,2025-07-25,20.7,72.8,0.010,12.2,1017.7,111.92,0.00,2.57,54.77,0.18,2.26,2.50,1.81,1
4,Paris,2025-07-26,21.4,67.6,0.000,16.0,1018.2,112.99,0.00,4.14,50.84,0.19,4.37,5.17,3.33,1
5,Paris,2025-07-27,18.8,78.8,5.600,23.2,1016.4,108.78,0.00,2.03,44.46,0.09,1.47,2.13,2.46,1
6,Paris,2025-07-28,18.9,65.9,0.000,13.8,1019.9,121.22,0.00,1.54,71.94,0.55,2.74,3.57,2.68,2
7,Berlin,2025-07-22,17.5,86.8,6.500,23.2,1006.1,114.07,0.00,5.65,38.48,0.78,2.47,2.77,1.76,1
8,Berlin,2025-07-23,18.4,83.8,27.700,25.0,1008.0,121.84,0.00,3.35,70.11,0.86,0.90,1.17,3.41,2
9,Berlin,2025-07-24,19.5,80.5,0.600,16.6,1011.3,113.37,0.00,4.48,69.40,1.01,2.21,2.68,5.15,2



## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [85]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
fig = px.density_heatmap(df_total,
                         title="Heatmap des températures par ville et jour",
                         x="datetime", 
                         y="city", 
                         z="temp", 
                         histfunc="avg")
fig.show()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()
fig = px.box(df_total,
            title="Comparatif des précipitations",
            x="city", 
            y="precip")
fig.show()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()
fig = px.scatter(df_total,
                 title="Qualité de l'air vs température",
                 x="temp", 
                 y="co",
                 trendline="ols")
fig.show()


1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

In [88]:
df_agg = df_total.groupby("city")["temp"].mean().reset_index()
df_agg.sort_values(by=["temp"], ascending=False, inplace=True)
fig = px.bar(df_agg,
             title="Températures moyennes par ville",
             y="temp",
             x='city')
fig.show()

In [96]:
df_mean = df_total.groupby("city")["precip"].mean().reset_index()
df_mean.sort_values(by=["precip"], ascending=False, inplace=True)
fig = px.bar(df_mean,
             title="Variabilité climatique",
             y="precip",
             x='city')
fig.show()

In [104]:
fig = px.scatter(df_total,
                         title="Corrélation température/humidité",
                         x="humidity",
                         y="temp",
                         trendline="ols")
fig.show()

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

In [133]:
fig = px.line(df_total,
              title="Evolution de la température sur 7 jours",
              x="datetime", 
              y="temp", 
              color="city")
fig.show()

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [181]:
fig = px.bar(df_total,
              title="Impact de la pluie sur la pollution",
              x="city",
              y="precip",
              color="indice aqi",
              barmode="overlay")
fig.show()

In [185]:
fig = px.histogram(df_total,
              title="Corrélation Vent/Qualité de l'air",
              x="city",
              y="windspeed",
              color="indice aqi",
              histfunc="avg",
              barmode="overlay")
fig.show()

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [52]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [53]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites